# But Why...?

Because it feels ridiculous! Also, if you can find complex patterns in multidimensional data can you find relatively simple patterns from a single input?

# Generating the Data

In [ ]:
import numpy as np

def fizz_buzz_encoder(n):
    '''
    Encode any given number to the FizzBuzz representation. That is 'fizz' if
    divisible by 3, 'buzz' if divisible by 5, 'fizzbuzz' if divisible by both
    3 and 5, or simply the number given. The encoded output is a vector 
    representing the output; [1,0] meaning a "fizz", [0,1] a "buzz", and [1,1]
    a "fizzbuzz". Note that the zero vector represents no word (just the 
    number).
    '''
    # Default to nothing in array
    vector = np.array([0,0])
    if n % 3 == 0: 
        vector += np.array([0,1])
    if n % 5 == 0:
        vector += np.array([1,0])
        
    return vector

def fizz_buzz_decoder(n_vector, n_default=''):
    '''
    Decodes the fizzbuzz vector representation to an output. See 
    fizz_buzz_encoder for details on the representation. n_default is the 
    default string representation if 'fizz', 'buzz' or 'fizzbuzz' should not
    be used in the string representation (usually just the number)
    '''
    # Use the vector to create the different 
    output = n_vector[0] * 'fizz' + n_vector[1] * 'buzz'
    # If zero vector, it should printout the default number
    if n_vector.sum() == 0:
        output = str(n_default)
    return output

## Produce data

In [ ]:
import pandas as pd

In [ ]:
# Test dataset
data = []

for number in np.arange(1,201):
    vector = fizz_buzz_encoder(number)
    output = fizz_buzz_decoder(vector,number)
    
    data.append({'n':number, 'fizz':vector[0], 'buzz':vector[1]})
    # print(f'{number:2}: {output:8} {vector}')

df_test.to_csv('fizzbuzz-data-test.csv')
df_test.head()

In [ ]:
# Train/Validate dataset
data = []

for number in np.arange(201,500102):
    vector = fizz_buzz_encoder(number)
    output = fizz_buzz_decoder(vector,number)
    
    data.append({'n':number, 'fizz':vector[0], 'buzz':vector[1]})
    # print(f'{number:2}: {output:8} {vector}')

df_test.to_csv('fizzbuzz-data-train-valid.csv')
df_test.head()

# A Simple Model

In [ ]:
import keras
from keras import models
from keras import layers
from keras import optimizers

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=1, name='input_layer'))
model.add(Dropout(0.2, name='input_dropout'))
# Add some complexity with more hidden layers
model.add(Dense(units=16, activation='relu', name='hl_complex'))
model.add(Dropout(0.2, name='hl_complex_dropout'))
# Attempt to embed whether or not it is divisible by 3 or 5
model.add(Dense(units=2, activation='relu', name='hl_embed'))
model.add(Dropout(0.2, name='hl_embed_dropout'))
# Classification
model.add(Dense(units=2, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True))

## Split into train-valid

In [ ]:
X = np.array([])
Y = []

for number in np.arange(201,100201):
    vector = fizz_buzz_encoder(number)
    X = np.append(X,number)
    Y.append(vector)
    
# For proper shape
Y = np.array(Y)

In [ ]:
# Scaling values for NN
X_scaled = (X - X.mean()) / (X.max() - X.min())

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(
    X_scaled, Y, test_size=0.2, random_state=27)

## Fit the model

In [ ]:
# Proof of concept run
model.fit(x_train, y_train, epochs=10, batch_size=16)